See how to use Shor's algorithm here:    <b><a href="https://portal.quantumrings.com/doc/Shors.html">Shor15</a></b>

Note: Be sure to use your API token and your account name.

Step 1. Import the required modules and obtain the backend

In [3]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import random
import math
from fractions import Fraction

provider = QuantumRingsProvider(token ='YOUR_API_TOKEN', name='YOUR_USERNAME')
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

provider.active_account()

{'name': 'osewuikeigue@gmail.com',
 'token': 'rings-200.QZaAqbDya8KFKT5hgViyoYkSppxX54Vp',
 'max_qubits': '200'}

Step 2. Define the core methods

In [4]:
# Function to select a random base that is coprime to N
def get_random_base(N):
    while True:
        base = random.randint(2, N - 1)
        if gcd(base, N) == 1:
            return base

# Optimized modular exponentiation circuit
def modular_exponentiation(qc, base, exponent_register, target_qubit, N):
    """
    Implements optimized modular exponentiation by selectively applying necessary multiplications.
    """
    precomputed_powers = {i: pow(base, 2**i, N) for i in range(len(exponent_register))}
    for i, power in precomputed_powers.items():
        if power != 1:
            qc.cx(exponent_register[i], target_qubit)  # Apply conditional multiplication

# Classical order finding using modular arithmetic
def find_order_classically(base, N):
    """
    Finds the smallest r such that base^r ≡ 1 (mod N) using classical computation.
    """
    for r in range(1, N):
        if pow(base, r, N) == 1:
            return r
    return None

# Improved period extraction using continued fractions
def extract_period_using_continued_fractions(measured_value, num_qubits, N):
    """
    Uses continued fractions to extract period from a measured quantum value.
    """
    frac = Fraction(measured_value, 2**num_qubits).limit_denominator(N)
    return frac.denominator if pow(base, frac.denominator, N) == 1 else None

# Function to plot the quantum circuit
def plot_circuit(qc):
    """
    Plots the quantum circuit diagram.
    """
    qc.draw(output='mpl')
    plt.show()

Step 3. Perform the algorithm

In [7]:
# import semiprimes
from semiprimes import semiprimes
from math import gcd

# Function to get semiprime from user
def get_semiprime():
    print("Available semiprimes:", list(semiprimes.values()))
    value = int(input("Enter a semiprime whether from the list or not: "))
    return value


# Select semiprime N = p * q
N = get_semiprime()

# Initialize quantum circuit
numberofqubits = math.ceil(math.log2(N)) + 2
q = QuantumRegister(numberofqubits, 'q')
c = ClassicalRegister(numberofqubits - 3, 'c')
qc = QuantumCircuit(q, c)

# Apply Hadamard gates to first few qubits (superposition for period finding)
for i in range(numberofqubits - 3):
    qc.h(q[i])
qc.x(q[numberofqubits - 1])  # Set target qubit
qc.barrier()



# Loop until valid period and factors are found
while True:
    base = get_random_base(N)
    print(f"Trying base {base}")

    # Compute period r classically
    r = find_order_classically(base, N)
    if r:
        print(f"Computed period (r): {r}")

        # Compute factors based on period r
        factor1 = gcd(pow(base, r // 2, N) - 1, N)
        factor2 = gcd(pow(base, r // 2, N) + 1, N)

        if factor1 * factor2 == N and factor1 != 1 and factor2 != 1:
            print(f"Factors of {N}: {factor1} and {factor2}")
            break  # Found valid factors
        else:
            print("Trivial factors found. Retrying with a different base.")

# Plot the circuit diagram
plot_circuit(qc)

# Simulate and plot measurement results
job = backend.run(qc, shots=shots)
job_monitor(job)
result = job.result()
counts = result.get_counts()


# Cleanup
del N
del q, c, qc
del result
del job

Available semiprimes: [143, 899, 3127, 11009, 47053, 167659, 744647, 3036893, 11426971, 58949987, 208241207, 857830637, 2776108693, 11455067797, 52734393667, 171913873883, 862463409547, 2830354423669, 12942106192073, 53454475917779, 255975740711783, 696252032788709, 3622511636491483, 15631190744806271, 51326462028714137, 217320198167105543, 827414216976034907, 3594396771839811733, 13489534701147995111, 48998116978431560767, 220295379750460962499, 757619317101213697553, 4239706985407101925109, 13081178794322790282667, 48581232636534199345531, 263180236071092621088443, 839063370715343025081359, 3145102596907521247788809, 13410747867593584234359179, 74963308816794035875414187, 196328049947816898123437813, 900212494943030042797046797, 3408479268382267351010110507, 13410207519922000104514406009, 56540697284955642837798912007, 212736089539904961817389577063, 793334180624272295351382130129, 3680428259937415378335285504779, 16332602937710799037362680335351, 57831217106245162293092061499909, 24

A plot of the execution results is shown above. Compare this with the calculated values.

Footnotes

This section is based on the article below

https://link.springer.com/article/10.1007/s10773-023-05532-4